# 📘 Week 3 Assignment Summary – Scalable & Secure Web Application on AWS

🛠️ **Note:** All necessary AWS resources and configurations (EC2, ALB, Auto Scaling Group, Security Groups, Launch Templates, and S3 buckets) were **pre-provisioned** in the assignment lab environment. Our role was to observe, validate behavior under load, and make targeted changes around **security (ports), cost, and scaling**.

---
## 📊 Architectural Overview

This week’s setup implements a **3-tier architecture**:

- **Data Tier**: Amazon S3 bucket holding the star schema output from the ETL pipeline  
- **Logic Tier**: Auto Scaling Group of EC2 instances behind an Application Load Balancer  
- **Presentation Tier**: A simple HTML dashboard displaying which EC2 instance served the request  

![Architectural Diagram](../images/archi_diagram.png)

The main objective of this assignment is to explore and implement a scalable, secure, and reliable web application architecture using AWS cloud services. The goal is to simulate how a real-world data application would serve analytical insights or dashboards to clients across the internet. This setup is designed to respond to client requests dynamically, with a focus on evaluating system behavior under varying loads, enforcing proper security, and improving cost efficiency.

When a user opens the website in their browser, an **HTTP request** is sent to the **Application Load Balancer (ALB)** — the central point that handles all incoming client traffic. The ALB then forwards the request to one of the **EC2 instances** in the **Auto Scaling Group**, based on load and availability. That EC2 instance processes the request, queries the transformed data stored in **Amazon S3**, and renders the output. The response is then sent back through the ALB to the user's browser over **port 80**, completing the round trip.

This architecture aligns with the **AWS Well-Architected Framework** by ensuring:
- 🔐 Secure access through restricted ports (only HTTP on port 80 is allowed),
- ⚙️ Reliable delivery using instances spread across **multiple Availability Zones**,
- 📈 Scalability through **Auto Scaling policies** triggered by request load,
- 💰 Cost efficiency via instance type optimization (e.g., switching to `t3.nano`).

By analyzing this design in action, we gain hands-on experience with cloud-native practices essential for deploying modern data-driven applications.

🧭 **Learn More**  
- 🌐 [Amazon EC2 Overview – AWS](https://aws.amazon.com/ec2/)  
- 🌐 [What is an Application Load Balancer – AWS](https://docs.aws.amazon.com/elasticloadbalancing/latest/application/introduction.html)  
- 🌐 [Auto Scaling Groups – AWS](https://docs.aws.amazon.com/autoscaling/ec2/userguide/AutoScalingGroup.html)  
- 🌐 [Amazon S3 – Object Storage](https://aws.amazon.com/s3/) 


---

## 🔍 Monitoring Web App Performance

We simulated user traffic using **Apache Benchmark (ab)** and used **Amazon CloudWatch** to monitor:

- CPU Utilization  
- Network In/Out  
- Auto Scaling Events  

We observed the EC2 metrics and scaling activities in the Monitoring tab.

![Monitoring Web App](../images/monitoring_web_app.png)

🧭 **Learn More**  
- ⚙️ [Apache Benchmark (ab) – Apache Docs](https://httpd.apache.org/docs/2.4/programs/ab.html)  
- 📊 [Amazon CloudWatch Overview – AWS](https://aws.amazon.com/cloudwatch/)  
---

## 🔐 Security Configuration

Initially, the ALB security group allowed traffic from **all IPs on *all* ports**, including a nonstandard **port 90** that exposed content we did not intend to make public.

To remediate:

- Removed the overly permissive rule (`All traffic`, `0.0.0.0/0`).  
- Added a rule allowing only **HTTP on port 80** from the internet (`0.0.0.0/0`).  
- (Optional best practice) Add HTTPS:443 later when TLS certs available.  
- Validated that traffic to **port 90** (and all other ports) was now **blocked**.  

![Security Diagram](../images/security.png)

### 🔐 Steps We Took to Lock Down Ports (Assignment Walkthrough)

1. **Identify Exposure** – Opened the ALB DNS URL with `:90` appended; page loaded → unintended access confirmed.  
2. **Review Security Group Rules** – Saw inbound `All traffic` from `0.0.0.0/0` attached to the ALB.  
3. **Remove Insecure Rule** – Deleted the open rule.  
4. **Add Least-Privilege Rule** – Created inbound rule: `HTTP | TCP 80 | 0.0.0.0/0`.  
5. **(Optional) Restrict Admin Ports** – If an internal admin port is needed, allow **only your IP / corporate CIDR**.  
6. **Test Closure** – Retry ALB DNS URL with `:90` → connection refused / timed out (expected).  
7. **Re-test Normal Access** – Plain `http://<ALB-DNS-Name>` still loaded web app over port 80.  
8. **Document Change** – Captured screenshots + justification for security audit.

📚 **Need a refresher on what a *port* is?** See: [What is a Port? (MDN)][mdn-port] • [IANA Service & Port Number Registry][iana-ports]

---

## ✅ Reliability via Availability Zones

The application was deployed across **multiple Availability Zones (AZs)** using Auto Scaling Groups:

- Each EC2 instance had a different internal IP and AZ.  
- Refreshing the page showed responses from different instances/AZs.  
- Results: **High availability** and **fault tolerance** if one AZ has issues.

![Reliability Diagram](../images/realibility.png)

---

## 💸 Cost Optimization

To optimize costs:

- Switched instance type from **t3.micro** to **t3.nano** in a new Launch Template version.  
- Updated the Auto Scaling Group to use the new template.  
- Terminated existing instances so the ASG launched right-sized ones.  

Outcome: Reduced compute cost while sustaining the assignment’s demo load.

![Cost Optimization](../images/cost_optimization.png)

---

## 📈 Scalability via Dynamic Auto Scaling

We created a dynamic scaling policy using the following settings:

- **Target metric**: ALB request count per target = 60  
- **Instance warm-up time**: 60 seconds  

**How it works:**

- If traffic spikes (example: 180 req/sec on 2 instances → 90 each), scaling triggers.  
- A new EC2 instance is launched.  
- Warm-up lets the instance initialize before being counted by scaling metrics.  

This filters noise and reduces “thrash scaling.”

We stress-tested with **1M Apache Benchmark requests** and observed:

- Additional EC2 instances launching.  
- Higher CPU + network usage in CloudWatch.  
- ASG scaling out, then (when load subsided) eligible to scale in.

![Scalability Diagram](../images/scalability.png)

---
## 🧠 Key Takeaways

- Implemented a **secure, cost-optimized, scalable** architecture using:  
  - S3, EC2, ALB, Security Groups, CloudWatch, and Auto Scaling  
- Practiced **least-privilege networking** (restrict ports; validate closure).  
- Built **fault-tolerant systems** with multi-AZ Auto Scaling.  
- Applied **FinOps** by right-sizing EC2 instances.  
- Configured **request-driven scaling** and observed it in action under load.  
- Understood how **DNS** and **ports** affect secure public access.

---

## 🔗 Reference Links

[mdn-port]: https://developer.mozilla.org/en-US/docs/Glossary/Port  
[iana-ports]: https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.xhtml  
[cloudflare-dns]: https://www.cloudflare.com/learning/dns/what-is-dns/  
[aws-dns-alb]: https://docs.aws.amazon.com/elasticloadbalancing/latest/application/load-balancer-fqdn-name.html